In [ ]:
import os
from pathlib import Path
from html import escape

HTML_HEADER = """\
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>All Mermaid Diagrams</title>
    <style>
        body    { font-family: sans-serif; margin: 0 2rem; }
        h2      { margin-top: 3rem; border-bottom: 1px solid #ddd; padding-bottom: 0.5rem; }
        .mermaid { margin: 2rem 0; }
    </style>
</head>
<body>
"""

HTML_FOOTER = """
<script type="module">
    import mermaid from 'https://cdn.jsdelivr.net/npm/mermaid@11/dist/mermaid.esm.min.mjs';
    mermaid.initialize({ startOnLoad: true });
  </script>
</body>
</html>
"""

def build_mermaid_gallery(root: Path = Path("."), output: str = "all_mermaid_diagrams.html",
                          allowed_ext: tuple[str, ...] = (".mmd", ".mermaid", ".txt", ".md")) -> None:
    """
    Walk `root`, read every file whose extension matches `allowed_ext`,
    and emit an HTML file containing a rendered Mermaid block for each file.
    """
    diagrams: list[str] = []

    for filepath in root.rglob("*"):
        if any(part.startswith(".ipynb_checkpoints") for part in filepath.parts):
            continue
        if filepath.is_file() and filepath.suffix.lower() in allowed_ext:
            try:
                mermaid_code = filepath.read_text(encoding="utf-8")
                # Optional: strip triple back-ticks in ```mermaid fenced blocks
                mermaid_code = mermaid_code.strip()
                if mermaid_code.startswith("```"):
                    mermaid_code = mermaid_code.lstrip("`").split("\n", 1)[1].rsplit("\n", 1)[0]

                relative = filepath.relative_to(root)
                diagrams.append(
                    f"<h2>{escape(str(relative))}</h2>\n"
                    f"<div class=\"mermaid\">\n{escape(mermaid_code)}\n</div>"
                )
            except Exception as exc:  # keep going even if one file fails
                print(f"⚠️  Skipped {filepath}: {exc}")
        

    if not diagrams:
        print("No matching files found – nothing to do.")
        return

    html = HTML_HEADER + "\n".join(diagrams) + HTML_FOOTER
    Path(output).write_text(html, encoding="utf-8")
    print(f"✅ Mermaid gallery written to {Path(output).resolve()}")

if __name__ == "__main__":
    build_mermaid_gallery()